## Expectations from this notebook

This notebook will read a KEP instance from a file, and then run a matching run on this instance using two different KEP schemas.

The first thing we need to do is install kep_solver, and import the relevant items. If you don't know what all of these are immediately, that's fine, we will go through them later.

In [ ]:
!pip install kep_solver
from kep_solver.fileio import read_json
from kep_solver.entities import BloodGroup
from kep_solver.programme import Programme
from kep_solver.model import TransplantCount, EffectiveTwoWay, BackArcs, ThreeWay, UKScore

We now read the given instance from a JSON file. File formats are described [online](https://kep-solver.readthedocs.io/en/latest/source/formats.html#input). Note that this sample input file is randomly generated. It is not representative of any particular population.

In [ ]:
instance = read_json("../tests/test_instances/medium-1.json")
print(f"This instance has {len(instance.recipients())} recipients with IDs: {', '.join(r.id for r in instance.recipients())}")

You can extract individual recipients and their paired donors and access information about them. For complete documentation on these entities see the [online documentation](https://kep-solver.readthedocs.io/en/latest/source/kep_solver.entities.html).

In [ ]:
recipient = instance.recipient("4")
print(f"Recipient {recipient.id} has {len(recipient.donors())} donor(s)")
first_donor = recipient.donors()[0]
print(f"Their first donor has blood group {first_donor.bloodGroup}")

You can also get all donors with `instance.donors()`, including non-directed donors.

In [ ]:
non_directed_donors = [donor for donor in instance.donors() if donor.NDD]
print(f"This instance has {len(non_directed_donors)} non-directed donors")

Now we can create a KEP. Our first KEP will be simple, with just one objective to maximise: the transplant count. We also configure the maximum cycle and chain lengths for this KEP, and then solve our instance. We get returned both the solution, and the model used to find the solution. The solution contains the `selected` exchanges, which we can examine and print out. Printing an exchange will list the donor, recipient, and vertex ID for each vertex (pair) in the transplant. For instance, `V(D_13,R_12 (8))` is a pair containing donor 13, recipient 12, and the (internal) vertex ID is 8.

In [ ]:
programme = Programme([TransplantCount()], description="My first KEP", maxCycleLength=3, maxChainLength=3)
solution, model = programme.solve_single(instance)
num_transplants = sum(len(modelled.exchange) for modelled in solution.selected)
print(f"A total of {len(solution.selected)} exchanges were found representing {num_transplants} transplants")
for t in solution.selected:
    print(str(t))

We now create a more complex KEP with multiple objectives. In particular, we replicate the current UK objectives as follows:
1. Maximise the number of effective two-way exchanges
2. Maximise the number of transplants
3. Maximise the number of back-arcs
4. Minimise the number of three-way exchanges
5. Maximise the score according to the UK calculations

Note that we still find same number of transplants, but the actual selected exchanges are different.

In [ ]:
objectives = [EffectiveTwoWay(), TransplantCount(), BackArcs(), ThreeWay(), UKScore()]
programme = Programme(objectives, description="UK KEP", maxCycleLength=3, maxChainLength=3)
solution, model = programme.solve_single(instance)
num_transplants = sum(len(modelled.exchange) for modelled in solution.selected)
print(f"A total of {len(solution.selected)} exchanges were found representing {num_transplants} transplants")
for t in solution.selected:
    print(str(t))

We can also examine the details of individual objectives, and also investigate how long it took to build and solve this model

In [ ]:
for objective, value in zip(objectives, solution.values):
    print(f"Calculated {objective.describe()} = {value}")
for description, time in solution.times:
    print(f"{description} took {time:.2f} seconds")